In [ ]:
#import

import torch
import torch.nn as nn # nn stuff
import torch.optim as optim # optimizer
import torch.nn.functional as f # no param functions exp, tanh vs keras functional
from torch.utils.data import DataLoader # utilities in data field
import torchvision.datasets as datasets # with tfds
import torchvision.transforms as transforms

In [ ]:
#create
class NN(nn.Module): # class inherit from Module class
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  def forward(self,x):
    x = f.relu(self.fc1(x))
    x = self.fc2(x)
    return x # compare to functional keras
model = NN(764, 10)
x = torch.randn(764, 764)
print(model(x).shape) # test out model (really just matrix mult so has shape)

torch.Size([764, 10])


In [ ]:
#set device
#conditional with .cuda.is_avaiable
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [ ]:
#hyperparam
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [ ]:
#load data
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform = transforms.ToTensor(),
                               download=True) # create folder , get train, download if not there
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
# batch - for batch in train_loader
#usually for i, batch in enumerate(train_loader, start=1) for index
# test on my own
test_dataset = datasets.MNIST(root='dataset/', train=False,
                              transform=transforms.ToTensor(),
                              download=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 88384303.57it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 25448675.17it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 26004088.66it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18567766.83it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [ ]:
#init
#tensor .to() -> device
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [ ]:
#loss and opt
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # learning rate
# Adam on model params

In [ ]:
#train
for epoch in range(num_epochs):
  for batch_ids, (data, targets) in enumerate(train_loader):
    # get data to cuda if pos
    data = data.to(device=device)
    targets = targets.to(device=device)

    #print(data.shape) #64, 1 for color, 28x28
    #set correct shape
    data = data.reshape(data.shape[0], -1)

    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad() # rezero
    loss.backward() # get losses # tf.backward similar

    optimizer.step() # steps through grad




In [ ]:
#eval#
def check_accuracy(loader, model):
  if loader.dataset.train: # check if train bool
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad(): # no any grads here
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1) # get high prob, gives index
      num_correct += (predictions == y).sum() # where indexes are correct
      num_samples += predictions.size(0)  # for batch
  print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f}')
  model.train()
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 55853 / 60000 with accuracy 93.09
Checking accuracy on test data
Got 9294 / 10000 with accuracy 92.94


In [ ]:
# CIFAR so far 100 of these set the bar.

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor


In [ ]:
my_layers_indices = [500, 250, 200, 150]
class CifarNN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(CifarNN, self).__init__(self)
    self.fc1 = nn.Linear(input_size, my_layers_indices)
    self.list_of_layers = [nn.Linear(my_layers_indices[i], my_layers_indices[i+1]) for i in range(len(my_layers_indices) - 1)]
    self.output = nn.Linear(my_layers_indices[len(my_layers_indices)-1], num_classes)
  def forward(self, inputs):
    x = f.relu(self.fc1(x))
    for layer in self.list_of_layers:
      x = f.relu(layer(x))
    return self.output(x)

In [ ]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#hyp
input_size = 3072
batch_size = 32
num_epochs = 36
num_classes = 100
lr = 0.001

In [ ]:
#load
train_dst = datasets.CIFAR100('datasets2/', train=True, download=True, transform=ToTensor())
train_loader = DataLoader(train_dst, shuffle=True, batch_size=batch_size)

test_dst = datasets.CIFAR100('datasets2/', train=False, download=True,  transform=ToTensor())
test_loader = DataLoader(test_dst, shuffle=True, batch_size=batch_size)

100%|██████████| 169001437/169001437 [00:02<00:00, 74741147.99it/s]


Extracting datasets2/cifar-100-python.tar.gz to datasets2/
Files already downloaded and verified


In [ ]:
#init - model init -> device
model = NN(input_size = input_size, num_classes = num_classes).to(device=device)

In [ ]:
#loss+opt
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.NAdam(model.parameters(), lr=lr)

In [ ]:
#train
num_epochs=5
for i in range(num_epochs):
  for j, (data_batch_data, data_batch_targets) in enumerate(train_loader):
    x = data_batch_data.to(device=device)
    y = data_batch_targets.to(device=device)

    x = x.reshape(x.shape[0], -1)
    scores = model(x)
    loss = criterion(scores, data_batch_targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

In [ ]:
def check_accuracy(loader,model ):
  num_correct = 0
  num_samples = 0
  if loader.dataset.train:
    print("training")
  else:
    print("testing")
  model.eval()
  for batch in loader:
    with torch.no_grad():
      x, y = batch
      x = x.to(device=device)
      y = y.to(device)

      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += x.size(0)
  print(float(num_correct)/float(num_samples))
  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

training
0.05256
testing
0.0493
